
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Advanced Pandas

<!-- ## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png)  -->

In this lesson you:
* Explore some more advanced features pandas provides including:
  - Renaming columns
  - Filtering the DataFrame
  - Grouping and aggregation Functions
  - Sorting
  - Imputing columns

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.
Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.
1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
    - In the drop-down, select **More**.
    - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.
**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
1. Find the triangle icon to the right of your compute cluster name and click it.
1. Wait a few minutes for the cluster to start.
1. Once the cluster is running, complete the steps above to select your cluster.


For this lesson, we are going to work with datasets. Running the cell below will define and give us access to variables defining the path to our datasets in the Databricks File System.

In [0]:
%run "./Includes/Classroom-Setup"

Remember, to access **`pandas`** functionality, we must **`import`** the library first. We do not have to **`pip install pandas`** because we already have it installed.

In [0]:
import pandas as pd

## Reading Data

<!-- <img src="https://files.training.databricks.com/images/301/sf.jpg" style="height: 200px; margin: 10px; border: 1px solid #ddd; padding: 10px"/> -->

So far we have created a DataFrame by manually specifying the rows and columns. Often, we will have a dataset stored as a CSV (comma-separated-value) file. 

**`pandas`** provides a function called [**read_csv(path)**](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html), where we provide a path to where our CSV file is stored, and it returns a DataFrame of the contents at that path.

You'll be analyzing data from <a href="http://insideairbnb.com/get-the-data.html" target="_blank">Inside Airbnb</a> to better understand the San Francisco rental market. Let's read in the dataset.

In [0]:
file_path = f"{DA.paths.datasets}/sf-airbnb/sf-airbnb.csv".replace("dbfs:", "/dbfs")
df = pd.read_csv(file_path)


To look at the first few records of the dataset, we can call [**head()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html). If you do not specify the number of rows, it defaults to 5 rows.

In [0]:
df.head(3)

Conversely, we can call [**tail()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html) to look at the last few records.

In [0]:
df.tail(3)

## Renaming Columns

We can rename columns of our DataFrame using [**rename()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html). We pass into columns a dictionary containing the mappings from the old column names to the new ones to the **`columns`** parameter.

Let's rename the **`neighbourhood`** column above to be **`neighborhood`**.

In [0]:
df = df.rename(columns={"neighbourhood": "neighborhood"})
df[["id", "neighborhood"]].head(10)

## Filtering

Often, you will want to select a subset of rows that meet a certain criteria, which can be accomplished by specifying: **`df[bool_array]`**, where **`bool_array`** is a **`Series`** of **`True`** and **`False`** values for each row. 

The rows that evaluate to **`True`** are kept, while the ones that evaluate to **`False`** are not. 

Let's filter for all the rows **`host_is_superhost`** is **`"t"`**, meaning the airbnb owner is a superhost.

In [0]:
filtered_df = df[df["host_is_superhost"] == "t"]
filtered_df[["id", "host_is_superhost"]].head(10)

Here, **`df["host_is_superhost"] == "t"]`** is our boolean array. Let's take a look at the corresponding **True/False** row indices.

In [0]:
df["host_is_superhost"] == "t"

We can also search for all the records where the **`host_is_superhost`** is NOT "t".

In [0]:
df["host_is_superhost"] != "t"

## Pandas Boolean Operators

Often you will want to evaluate multiple criteria to filter out records. For example, let's select all records where the host is a superhost and the airbnb has at least 150 reviews.

Instead of the normal Boolean operators we have seen previously, we have [bitwise Boolean operators](https://www.w3schools.com/python/gloss_python_bitwise_operators.asp):
* **`and`** -> **`&`**
* **`or`** -> **`|`** 
* **`not`** -> **`~`**

In [0]:
filtered_df = df[(df["host_is_superhost"] == "t") & (df["number_of_reviews"] >= 150)]
filtered_df[["id", "host_is_superhost", "number_of_reviews"]].head(10)

## Aggregate Functions

Aggregate functions are functions that take in a series of inputs and return a single output. 

The most common ones that we use in pandas are ones that take in numerical **`Series`** and return a statistic of interest, such as the mean. 

Let's take a look at the mean, min, and max of **`number_of_reviews`**:

In [0]:
print(df["number_of_reviews"].mean())
print(df["number_of_reviews"].min())
print(df["number_of_reviews"].max())

Another useful method is [**describe()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) which provides a report of summary statistics on a given numerical **`Series`**:

In [0]:
df["number_of_reviews"].describe()

We can also use this method on a DataFrame to see it applied to every numerical column:

In [0]:
df[["number_of_reviews", "host_listings_count", "bedrooms"]].describe()

Many times, you won't care about the 6th value after the decimal. Let's round our results by calling [**round()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html?highlight=round#pandas.DataFrame.round).

In [0]:
df[["number_of_reviews", "host_listings_count", "bedrooms"]].describe().round(2)

## Group By

Sometimes we will want to see the results of an aggregate function per category in a non-numerical column. 

For example, say we wanted to see the average number of bedrooms per neighborhood.

In order to do this we first use the [**groupby([columns])**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) method and specify the category we want to group by. In this case, let's group by **`neighborhood`**.

In [0]:
df.groupby(["neighborhood"])

Then we apply the aggregate function of interest. In this case, **`mean()`** to the column of interest, in this case **`bedrooms`**.

**Note:** Here we use **`[["bedrooms"]]`** to select for bedrooms because we could add other columns in addition to bedrooms.

In [0]:
grouped_df = df.groupby(["neighborhood"])[["bedrooms"]].mean().head(10)
grouped_df

# Reset Index

DataFrames always have some sort of index, which we've seen displayed as the leftmost column in the examples above. By default, they are numbers, but many operations can change those indices to something else. In the example above, **`neighborhood`** became the index rather than a column. We can see that if we print out the columns.

In [0]:
grouped_df.columns

Sometimes, resetting the index back to its default integer sequence is desirable. Some cases where this might be useful include:
* Eliminating duplicates in the current index
* Transforming the current index into a column so that you can include those values in columnar computations or transformations
* Making the index consistent and contiguous following an operation that altered the DataFrames shape or structure
 
To reset the index, use [**reset_index()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html).

In [0]:
reset_df = grouped_df.reset_index()
reset_df

 
## Sorting

Pandas provides a [**sort_values()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) method to sort the rows in a **`DataFrame`** or **`Series`**.

If called on a **`DataFrame`** you need to specify which column you are sorting by like this **`df.sort_values([col])`**

In [0]:
sorted_df = df.sort_values(["bedrooms"])
sorted_df[["id","bedrooms"]].head(10)

If applied to a **`Series`** there is only one column, so you don't need to specify:

In [0]:
df["bedrooms"].sort_values()

By default **`sort_values()`** sorts in ascending order. You can specify the **`ascending=False`** parameter to change it to descending order.

In [0]:
df["bedrooms"].sort_values(ascending=False)

 
# NaN 

You might have noticed that our **`DataFrame`** contains NaN values. These indicate a missing value. 

We have a few ways we can handle missing values. Often having these values present causes problems for computational tasks.

First, we can check using the [**isna()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html#pandas.DataFrame.isna) method, alias for [**isnull()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html#pandas.DataFrame.isnull), and the **`sum()`** method to count the number of NaN values present.

In [0]:
nan_df = df[["security_deposit", "notes"]] # subset of columns with NaNs
nan_df

In [0]:
nan_df.isna().sum()

## Dropping NaN

One way you can handle NaN is to drop all rows that have NaN values. We can use the [**dropna()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) method to do that.

In [0]:
nan_df.dropna()

### Impute Columns

However, we are throwing away a lot of information when we drop records - in the example above, we removed over 3000 rows.

Instead of dropping rows with missing values, we can impute the missing values using [**fillna()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) and specifying a default value to use.

In [0]:
nan_df.fillna("Missing")

Oftentimes, we want to impute different values to different columns. For example, with `numeric` values, we can impute with the mean/median/etc. For `categorical` features, imputing with the mode or a special category are common.

Let's instead specify that **`security_deposit`** is `$0.00` if it is missing. We can pass in a dictionary to **`fillna()`** that has column names as the key and the value to impute the column with as the value. 

You can optionally specify **`inplace=True`** if you want to update the underlying DataFrame.

In [0]:
nan_df.fillna({"security_deposit": "$0.00", "notes": "Missing"}, inplace=False)

 
## Write to CSV

We can write a **`pandas`** DataFrame to a CSV file as shown below using the [**to_csv()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) method.

In [0]:
file_path = DA.paths.working_dir.replace("dbfs:", "/dbfs") + ".csv"
df.to_csv(file_path, index=False)

 
We can then read our csv file back in with **`read_csv`**.

In [0]:
load_df = pd.read_csv(file_path)
load_df.head()

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>